In [1]:
from gymnasium import spaces
import yaml
import torch

from agilerl.modules.configs import MlpNetConfig, CnnNetConfig, MultiInputNetConfig
from agilerl.networks.q_networks import QNetwork, RainbowQNetwork
from agilerl.networks.value_functions import ValueFunction
from agilerl.networks.actors import StochasticActor, DeterministicActor

from agilerl.algorithms.dqn import DQN
from agilerl.utils.utils import create_population

/home/jaimesabal/.pyenv/versions/agilerl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### QNetwork

In [2]:
from tests.helper_functions import generate_dict_or_tuple_space
from agilerl.utils.evolvable_networks import is_image_space

img_space = spaces.Box(low=0, high=255, shape=(4, 84, 84))
vec_space = spaces.Box(low=-1, high=1, shape=(4,), dtype='float32')
dict_space = spaces.Dict({'img': img_space, 'vec': vec_space})
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

img_config = CnnNetConfig(
    channel_size=[16],
    kernel_size=[4],
    stride_size=[1],
)
vec_config = MlpNetConfig(
    hidden_size=[64],
)
multi_input_config = MultiInputNetConfig(
    channel_size=[8, 8, 8],
    kernel_size=[2, 2, 2],
    stride_size=[2, 2, 2],
    hidden_size=[32, 32, 32],
    vector_space_mlp=False
)

actor = RainbowQNetwork(
    observation_space=vec_space,
    action_space=spaces.Discrete(4),
    support=torch.linspace(-10, 10, 51),
    latent_dim=64,
    device=device
)

In [ ]:
from numbers import Number
from dataclasses import dataclass, field
from typing import Any, Dict, Optional

@dataclass
class RLHyperparameter:
    min: float
    max: float
    shrink_factor: float = 0.8
    grow_factor: float = 1.2
    dtype: type = float
    value: Optional[Number] = field(default=None, init=False)

    def mutate(self) -> Number:
        assert self.value is not None, "Hyperparameter value is not set"

        # Equal probability of growing or shrinking 
        if torch.rand(1).item() < 0.5:
            if self.value * self.shrink_factor > self.min:
                self.value *= self.shrink_factor
            else:
                self.value = self.min
        else:
            if self.value * self.grow_factor < self.max:
                self.value *= self.grow_factor
            else:
                self.value = self.max

        return self.value

class HyperparameterConfig:
    """Stores the RL hyperparameters that will be mutated during training. For each
    hyperparameter, we store the name of the attribute where the hyperparameter is 
    stored, and the range of values that the hyperparameter can take."""

    def __init__(self, **kwargs: Dict[str, RLHyperparameter]):
        self.config = kwargs
        for key, value in kwargs.items():
            setattr(self, key, value)
    
    def items(self) -> Dict[str, Any]:
        return self.config.items()
    

HyperparameterConfig(lr=(1e-3, 1e-2), learn_step=(1000, 3000)).items()

In [3]:
from agilerl.modules.cnn import EvolvableCNN
from agilerl.hpo.mutation import Mutations

In [8]:
from accelerate import Accelerator

with open('configs/training/ppo.yaml') as f:
    config = yaml.safe_load(f)

vector_actions = spaces.Box(low=-1, high=1, shape=(4,), dtype='float32')
discrete_actions = spaces.Discrete(4)

# accelerator = Accelerator()
INIT_HP = config["INIT_HP"]
INIT_HP['AGENT_IDS'] = [f'agent_{i}' for i in range(4)]
n_agents = 4
agent_pop = create_population(
    algo=INIT_HP["ALGO"],
    observation_space=img_space,
    action_space=discrete_actions,
    net_config={'encoder_config': img_config},
    INIT_HP=INIT_HP,
    population_size=INIT_HP["POP_SIZE"],
    num_envs=INIT_HP["NUM_ENVS"],
    device=device,
    # accelerator=accelerator
)

In [9]:
ind = agent_pop[0]
print(ind.lr)
print(ind.learn_step)
print(ind.batch_size)

0.001
2048
128
